In [ ]:
''' Regrid AMSR2 data to rectilinear grid '''

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import numpy.ma as ma
import xarray as xr
import xesmf as xe
import pyresample
import pandas as pd
import glob
import fnmatch
import regionmask
from datetime import datetime
from area import area

In [ ]:
def polar_xy_to_lonlat(x, y, true_scale_lat, re, e, hemisphere, lon_0):
    """Convert from Polar Stereographic (x, y) coordinates to
    geodetic longitude and latitude.
    Args:
        x (float): X coordinate(s) in km
        y (float): Y coordinate(s) in km
        true_scale_lat (float): true-scale latitude in degrees
        hemisphere (1 or -1): 1 for Northern hemisphere, -1 for Southern
        re (float): Earth radius in km
        e (float): Earth eccentricity
    Returns:
        If x and y are scalars then the result is a
        two-element list containing [longitude, latitude].
        If x and y are numpy arrays then the result will be a two-element
        list where the first element is a numpy array containing
        the longitudes and the second element is a numpy array containing
        the latitudes.
    """

    e2 = e * e
    slat = true_scale_lat * np.pi / 180
    rho = np.sqrt(x ** 2 + y ** 2)

    if abs(true_scale_lat - 90.) < 1e-5:
        t = rho * np.sqrt((1 + e) ** (1 + e) * (1 - e) ** (1 - e)) / (2 * re)
    else:
        cm = np.cos(slat) / np.sqrt(1 - e2 * (np.sin(slat) ** 2))
        t = np.tan((np.pi / 4) - (slat / 2)) / \
            ((1 - e * np.sin(slat)) / (1 + e * np.sin(slat))) ** (e / 2)
        t = rho * t / (re * cm)

    chi = (np.pi / 2) - 2 * np.arctan(t)
    lat = chi + \
        ((e2 / 2) + (5 * e2 ** 2 / 24) + (e2 ** 3 / 12)) * np.sin(2 * chi) + \
        ((7 * e2 ** 2 / 48) + (29 * e2 ** 3 / 240)) * np.sin(4 * chi) + \
        (7 * e2 ** 3 / 120) * np.sin(6 * chi)
    lat = hemisphere * lat * 180 / np.pi
    lon = np.arctan2(hemisphere * x, -hemisphere * y)
    lon = hemisphere * lon * 180 / np.pi
    lon = lon + lon_0
    lon = lon + np.less(lon, 0) * 360
    return [lon, lat]

def find_amsr2(date,indir):
    '''Find AMSR2 file by date. For .nc files only.
    Args:
        date (string): 'YYYYMMDD'
        indir (string): local path to directory conntaining AMSR2 files
    '''
    if indir == None:
        indir = '/local/jbj13rpu/Datasets/amsr2/'
    f_list_all = []
    for f in glob.iglob(indir + '**', recursive=True):
        f_list_all.append(f)
    result = []

    pattern = '*' + date + '*' + '.nc'

    for f in f_list_all:
        if fnmatch.fnmatch(f,pattern):
            result.append(f)
    if len(result) != 1:
        print('Warning, more than one AMSR2 file found!')
    result = result[0]
    return result

In [ ]:
indir = '/local/jbj13rpu/Datasets/amsr2/'
date_range = pd.date_range('2018-02-01T12:00:00',
                          '2018-03-31T12:00:00',freq='24H')
amsr2_list = []

for datetime in date_range:
    date_str = pd.to_datetime(date_range[0]).strftime("%Y%m%d")
    f = find_amsr2(date_str,indir)
    ds_amsr2 = xr.open_dataset(f)
    ds_amsr2 = ds_amsr2.assign_coords({'time': datetime})
    ds_amsr2 = ds_amsr2.expand_dims('time')
    amsr2_list.append(ds_amsr2)

In [ ]:
# open an individual AMSR2 file to set original curvilinear grid
ds = amsr2_list[0]
### calculate 2D lat/lon arrays from from curvilinear y/x grid
# TO DO - turn this into reusable function

x = ds.x
y = ds.y

# set parameters for projection
true_scale_lat = 70
re = 6378237 / 1000 #m to km
# e = math.sqrt(6.69437999014*10**-3)
e = 0.081816153
hemisphere = 1 # North = 1
lon_0 = -45

# must convert x/y from m to km
xc,yc = polar_xy_to_lonlat(x/1000, y/1000, 
                           true_scale_lat, re,
                           e, hemisphere, lon_0)

# add lat/lon to ds
ds = ds.assign_coords({'lon': xc, 'lat': yc})

### set output grid
# resolution in degrees
res = 0.06 # ~6 km
ds_out = xe.util.grid_global(res,res)

### set regridder (time intensive step, may kill kernel at high res, reuse file where poss)
regridder_amsr2 = xe.Regridder(ds, ds_out, 'bilinear',
                        reuse_weights='True')

In [ ]:
# perform regridding
amsr2_regridded_list = []

for ds in amsr2_list:
    # regrid
    da_in = ds['z']
    da_in = da_in.transpose()
    time = da_in.time.data
    da_out = regridder_amsr2(da_in.isel(time=0))
    
    # collect data
    lon_1d = da_out.lon[0,:].data
    lat_1d = da_out.lat[:,0].data
    sic = da_out.data

    # create new clean dataarray
    da_amsr2 = xr.DataArray(sic,coords={'lat':lat_1d,'lon':lon_1d},
                            dims=['lat','lon'])
    ds_amsr2 = da_amsr2.to_dataset(name='siconc')
    ds_amsr2 = ds_amsr2.assign_coords({'time': time})
    
    # list arrays
    amsr2_regridded_list.append(ds_amsr2)
    
# merge into single output file
for i in range(len(amsr2_regridded_list)):
    if i == 0:
        ds_amsr2_large = amsr2_regridded_list[0].copy()
    if i > 0:
        ds_amsr2_large = ds_amsr2_large.merge(amsr2_regridded_list[i])
        
ds_amsr2_large.attrs['data'] = 'AMSR2 sea ice concentration'
ds_amsr2_large.attrs['author'] = 'Chris Barrell [chrisbarrell16@gmail.com]'
ds_amsr2_large.attrs['time_created'] = str(datetime.now())
ds_amsr2_large.attrs['source'] = 'University of Bremen, Gunnar Spreen [gunnar.spreen@uni-bremen.de], Christian Melsheimer [melsheim@uni-bremen.de]: AMSR2 sea ice concentration based on the ASI algorithm (Spreen et al., 2008)'
ds_amsr2_large.attrs['regrdding'] = 'Regridded from polar stereographic using xesmf to 0.06 deg regular lat lon grid'

In [ ]:
ds_amsr2_large.to_netcdf('/local/jbj13rpu/Datasets/amsr2_regridded/AMSR2_6km_during_IGP.nc')